# Lectura y balanceo de datos

## Lectura

In [1]:
#Paquetes
import tensorflow as tf
import numpy as np

#Extracción de datos
lote = 20000 #Numero de muestras con el que se trabaja hasta que se actualizan los parametros
data_training = tf.keras.preprocessing.text_dataset_from_directory( #Se necesita TensorFlow version 2.3.0 o superior
    "aclImdb/train", #Ruta de los datos
    batch_size=lote, #Tamaño del lote
    validation_split=0.2, #Proporcion de muestras de validacion
    subset="training", #Tipo de subset
    seed=1337, #Semilla desde la que empiezo a coger muestras
)
data_validation = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=lote-15000,
    validation_split=0.2,
    subset="validation",
    seed=1337,
)
data_test = tf.keras.preprocessing.text_dataset_from_directory(
    "aclImdb/test", batch_size=lote + 5000
)

#Número de lotes
print(
    "Número de lotes de entrenamiento: %d"
    % tf.data.experimental.cardinality(data_training)
)
print(
    "Número de lotes de validación: %d" % tf.data.experimental.cardinality(data_validation)
)
print(
    "Número de lotes de prueba: %d"
    % tf.data.experimental.cardinality(data_test))

In [2]:
#Bucle para extraer los valores
for texto, etiqueta in data_training.take(1):
    for i in range(5):
        print('Texto: ',texto.numpy()[i])
        print('Etiqueta: ',etiqueta.numpy()[i])

# Preparación del texto

In [42]:
#Paquetes
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import string
import re

# Creamos una función para eliminar las / de nuestro texto, y poner todas las letras en minúsculas
def preparacion(texto):
    texto_minusculas = tf.strings.lower(texto)
    sin_barras = tf.strings.regex_replace(texto_minusculas, "<br />", " ")
    return tf.strings.regex_replace(
        sin_barras, "[%s]" % re.escape(string.punctuation), "")


# Definimos la vectorización.
caracteristicas = 20000 #Número máximo de tokens de la capa (con cuantas palabras trabaja)
long_seq = 500 #Longitud máxima de cada secuencia

# Definimos la capa de vectorizacion
capa_vectorizacion = TextVectorization(
    #Definimos los valores
    standardize=preparacion,
    max_tokens=caracteristicas,
    output_mode="int", #El vector que sale es de enteros
    output_sequence_length=long_seq,
)


In [43]:
def vectorizacion(texto, etiqueta):
    texto = tf.expand_dims(texto, -1)
    return capa_vectorizacion(texto), etiqueta


# Vectorize the data.
training = data_training.map(vectorizacion)
validation = data_validation.map(vectorizacion)
test = data_test.map(vectorizacion)

In [3]:
#Volvemos a extraer los valores con el mismo bucle
for texto, etiqueta in training.take(1):
    for i in range(5):
        print('Texto: ',texto.numpy()[i])
        print('Etiqueta: ',etiqueta.numpy()[i])

## Balanceo 

In [4]:
#Primero pasamos los arrays a numpy como estamos acostumbrados
for texto, etiqueta in training.take(1):
    dataTraining = texto.numpy()
    targetTraining = etiqueta.numpy()

for texto, etiqueta in validation.take(1):
    dataValidation = texto.numpy()
    targetValidation = etiqueta.numpy()
    
for texto, etiqueta in test.take(1):
    dataTest = texto.numpy()
    targetTest = etiqueta.numpy()
    
Después contamos los valores positivos y negativos que tengo en el entrenamiento

#Inicializacion de la cuenta
Positivo = 0
Negativo = 0

#Cuenta de datos
for x in range(targetTraining.shape[0]):
    if targetTraining[x] == 0:
        Negativo = Negativo + 1
    else:
        Positivo = Positivo + 1

#Representación
print("Reseñas positivas: " + str(Positivo))
print("Reseñas negativas: " + str(Negativo))

In [5]:
dataTraining.shape

In [6]:
#Importamos los paquetes de subremuestreo
from imblearn.under_sampling import NearMiss

#SMOTE
NM = NearMiss()

#Generación de nuevas muestras sintéticas
dataTrainingNM, tragetTrainingNM = NM.fit_resample(dataTraining, targetTraining)

#Inicializacion de la cuenta
Positivo = 0
Negativo = 0

#Cuenta de datos
for x in range(tragetTrainingNM.shape[0]):
    if tragetTrainingNM[x] == 0:
        Negativo = Negativo + 1
    else:
        Positivo = Positivo + 1

#Representación
print("Reseñas positivas: " + str(Positivo))
print("Reseñas negativas: " + str(Negativo))